Specific game:
- https://api.rawg.io/api/games/rimworld
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

## Get info of each game

In [1]:
import csv
import requests
import json
from pprint import pprint
from time import time
import concurrent.futures
import functools

In [2]:
csv_data = []
with open("data/game_id.csv", "r") as f:
    csv_data = list(csv.reader(f))
for i, val in enumerate(csv_data):
    print(val)
    if i==10: break

['3498', 'grand-theft-auto-v']
['4200', 'portal-2']
['3328', 'the-witcher-3-wild-hunt']
['12020', 'left-4-dead-2']
['5679', 'the-elder-scrolls-v-skyrim']
['802', 'borderlands-2']
['13536', 'portal']
['3439', 'life-is-strange-episode-1-2']
['4291', 'counter-strike-global-offensive']
['4062', 'bioshock-infinite']
['1030', 'limbo']


In [16]:
def threading(start_index, games_per_worker, urls):
    for url in urls[start_index : start_index + games_per_worker]:
#         try:
            # Get json reqquest
            json_data = json.loads(requests.get(url, headers=headers).text)

            # Only include wanted keys
            D = {k:v for k,v in json_data.items() if k in include}

            # Clean up dictionary
            D["platforms"] = []
            for platform in json_data["platforms"]:
                D["platforms"].append(platform["platform"]["name"])

            for key in ("developers", "genres", "publishers"):
                D[key] = []
                for data in json_data[key]:
                    D[key].append(data["name"])

            if json_data["esrb_rating"]:
                D["esrb_rating"] = json_data["esrb_rating"]["name"]

            name = D["id"]
            with open(f"data/game_info/{name}.json","w", encoding="utf-8") as f:
                json.dump(D, f)
#         except:
#             print(f"Error with {url}")

## Multithreading

#### Preparation

In [21]:
headers = { 'User-Agent': 'App Name: Education purpose',}
include = {"id",
           "slug",
           "name",
           "metacritic",
           "released",
           "tba",
           "updated",
           "website",
           "rating",
           "rating_top",
           "added_by_status",
           "playtime",
           "achievements_count",
           "ratings_count",
           "suggestions_count",
           "game_series_count",
           "reviews_count",
           "platforms",
           "developers",
           "genres",
           "publishers",
           "esrb_rating",
           }

max_workers = 100
start_game_index = 100000 # min: 0
end_game_index = 110000 # max: 345707
number_of_games = end_game_index - start_game_index
games_per_worker = int(number_of_games/max_workers) 
start_index = range(start_game_index, end_game_index, games_per_worker)

# Make urls
base_url = "https://api.rawg.io/api/games/"
urls = [base_url + csv_data[i][0] for i in range(len(csv_data))]

In [22]:
print(*start_index)

100000 100100 100200 100300 100400 100500 100600 100700 100800 100900 101000 101100 101200 101300 101400 101500 101600 101700 101800 101900 102000 102100 102200 102300 102400 102500 102600 102700 102800 102900 103000 103100 103200 103300 103400 103500 103600 103700 103800 103900 104000 104100 104200 104300 104400 104500 104600 104700 104800 104900 105000 105100 105200 105300 105400 105500 105600 105700 105800 105900 106000 106100 106200 106300 106400 106500 106600 106700 106800 106900 107000 107100 107200 107300 107400 107500 107600 107700 107800 107900 108000 108100 108200 108300 108400 108500 108600 108700 108800 108900 109000 109100 109200 109300 109400 109500 109600 109700 109800 109900


In [24]:
urls[100000:100010]

['https://api.rawg.io/api/games/114090',
 'https://api.rawg.io/api/games/114089',
 'https://api.rawg.io/api/games/114088',
 'https://api.rawg.io/api/games/114087',
 'https://api.rawg.io/api/games/114086',
 'https://api.rawg.io/api/games/114085',
 'https://api.rawg.io/api/games/114084',
 'https://api.rawg.io/api/games/114083',
 'https://api.rawg.io/api/games/114082',
 'https://api.rawg.io/api/games/114081']

In [15]:
# Time
t0 = time()

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(threading, games_per_worker=games_per_worker, urls=urls)
    executor.map(temp, start_index)
    
    
# Time diff
print(time()-t0)

6.20949912071228
